In [4]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader
from tqdm import tqdm
import os
from sklearn.metrics import classification_report, f1_score
from torch.utils.data import Dataset
import collections
import emoji
import re
from emot.emo_unicode import EMOTICONS_EMO
import nltk
nltk.download('words')
import nltk, string, re, spacy,unicodedata, random
nltk.download('punkt')
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.tokenize import ToktokTokenizer

[nltk_data] Downloading package words to
[nltk_data]     C:\Users\cs\AppData\Roaming\nltk_data...
[nltk_data]   Package words is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\cs\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [9]:
!pip install transformers

  Using cached transformers-4.27.4-py3-none-any.whl (6.8 MB)


In [2]:
!pip install emot 
!pip install emoji 
!pip install nltk 
!pip install -U spacy 
!pip install sentencepiece

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [5]:
def deEmojify(string):
    emoji_pattern = re.compile("["
                              u"\U0001F600-\U0001F64F"  # emoticons
                              u"\U0001F300-\U0001F5FF"  # symbols & pictographs
                              u"\U0001F680-\U0001F6FF"  # transport & map symbols
                              u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                              u"\U00002500-\U00002BEF"  # chinese char
                              u"\U00002702-\U000027B0"
                              u"\U00002702-\U000027B0"
                              u"\U000024C2-\U0001F251"
                              u"\U0001f926-\U0001f937"
                              u"\U00010000-\U0010ffff"
                              u"\u2640-\u2642"
                              u"\u2600-\u2B55"
                              u"\u200d"
                              u"\u23cf"
                              u"\u23e9"
                              u"\u231a"
                              u"\ufe0f"  # dingbats
                              u"\u3030"
                              "]+", flags=re.UNICODE)
    return emoji_pattern.sub(r'', string)
        
def preprocess(text):
    text = str(text)
    text = deEmojify(text) #remove emojis
    text = re.sub(r'([\.\'\"\/\-\_\--])',' ', text) # remove punctuations , removes @USER / some abbreviatins
    to_remove_url = ('http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|'
      '[!*,]|(?:%[0-9a-fA-F][0-9a-fA-F]))+')
    text = re.sub(to_remove_url,'',text)  # remove url patterns
    text = re.sub(" \d+", " ", text)
    text = text.replace(","," ")
    text = re.sub(r'(?:^| )\w(?:$| )', ' ', text).strip()
    punctuation='!!"$%&()*+-/:;<=>?[\]^_{|}~.'
    text = ''.join(ch for ch in text if ch not in set(punctuation))
    
    tokenizer = ToktokTokenizer()
    # convert sentence into token of words
    tokens = tokenizer.tokenize(text)
    tokens = [token.strip() for token in tokens]
    text = ' '.join(ch for ch in tokens)
    return text 

def clean(df):
    df['text'] = df['text'].apply(lambda x: preprocess(x))


In [6]:
train = pd.read_csv('tamil_train.csv')
val = pd.read_csv('tamil_dev.csv')
train.drop(['Unnamed: 0'], axis=1, inplace=True)
val.drop(['Unnamed: 0'], axis=1, inplace=True)
train.dropna()
train['label'] = pd.Categorical(train.label)
val.dropna()
val['label'] = pd.Categorical(val.label)
train['text'].apply(str)
clean(train)
clean(val)

In [7]:
class tamil_Offensive_Dataset(Dataset):
    def __init__(self, encodings, labels, bpe = False):
        self.encodings = encodings
        self.labels = labels
        self.is_bpe_tokenized = bpe

    def __getitem__(self, idx):
        if not self.is_bpe_tokenized:
            item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        else:
            item = {
                'input_ids': torch.LongTensor(self.encodings[idx].ids),
                'attention_mask': torch.LongTensor(self.encodings[idx].attention_mask)
            }
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)

In [10]:
from transformers import GPT2LMHeadModel, GPT2Tokenizer, GPT2ForSequenceClassification
tokenizer = GPT2Tokenizer.from_pretrained('sberbank-ai/mGPT')
model = GPT2ForSequenceClassification.from_pretrained('sberbank-ai/mGPT',num_labels = 3)
model_name = "sparse_attention_model"

Some weights of the model checkpoint at sberbank-ai/mGPT were not used when initializing GPT2ForSequenceClassification: ['lm_head.weight']
- This IS expected if you are initializing GPT2ForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing GPT2ForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of GPT2ForSequenceClassification were not initialized from the model checkpoint at sberbank-ai/mGPT and are newly initialized: ['score.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [11]:
tokenizer.padding_side = "left"
tokenizer.pad_token = tokenizer.eos_token

model.resize_token_embeddings(len(tokenizer))
model.config.pad_token_id = model.config.eos_token_id

In [13]:
 # Optimiser
from transformers import AdamW
optimizer = AdamW(model.parameters(), lr=1e-5)

In [14]:
label_mapping = {
        'Not_offensive': 0, 
        'not-Tamil': 1, 
        'Offensive': 2, 
    }

In [15]:
# Collecting Text and Labels
train_batch_sentences = list(train['text'])
train_batch_labels =  [label_mapping[x] for x in train['label']]
dev_batch_sentences = list(val['text'])
dev_batch_labels =  [label_mapping[x] for x in val['label']]

In [16]:
# Convert to Tensor
if 'parameters' in tokenizer.__dict__.keys() and tokenizer.__dict__['_parameters']['model'] == 'ByteLevelBPE':
    train_encodings = tokenizer.encode_batch(train_batch_sentences)
    dev_encodings = tokenizer.encode_batch(dev_batch_sentences)
else:
    train_encodings = tokenizer(train_batch_sentences, padding='max_length', truncation=True, max_length=64, return_tensors="pt")
    dev_encodings = tokenizer(dev_batch_sentences, padding='max_length', truncation=True, max_length=64, return_tensors="pt")

train_labels = torch.tensor(train_batch_labels)
dev_labels = torch.tensor(dev_batch_labels)

In [17]:
# Defining Datasets
train_dataset = tamil_Offensive_Dataset(train_encodings, train_labels, bpe = False)
dev_dataset = tamil_Offensive_Dataset(dev_encodings, dev_labels, bpe = False)

device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
model.to(device)
best_val_f1 = 0
count = 0

In [18]:
# Alternate Loss Fn
# Weighted Manual Loss Function
from sklearn.utils import class_weight
import torch.nn as nn

weights = class_weight.compute_class_weight(class_weight = 'balanced',classes = np.unique(train_batch_labels),y = train_batch_labels)
weights = np.exp(weights)/np.sum(np.exp(weights))
class_weights = torch.FloatTensor(weights).to(device)
loss_function = nn.CrossEntropyLoss(weight=class_weights, reduction='mean')

In [19]:
# Dataloaders
train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)
dev_loader = DataLoader(dev_dataset, batch_size=16, shuffle=False)
loss_weighted = True

In [ ]:
for epoch in range(30):
    train_preds = []
    train_labels = []
    total_train_loss = 0
    model.train()
    print("==========================================================")
    print("Epoch {}".format(epoch))
    print("Train")
    for batch in tqdm(train_loader):
        optimizer.zero_grad()
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['labels'].to(device)
        outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
        if loss_weighted:
            loss = loss_function(outputs[1], labels)
        else:
            loss = outputs[0]
        loss.backward()
        optimizer.step()

        for logits in outputs[1].detach().cpu().numpy():
            train_preds.append(np.argmax(logits))
        for logits in labels.cpu().numpy():
            train_labels.append(logits)
        total_train_loss += loss.item()/len(train_loader)

    print("Dev")
    dev_preds = []
    model.eval()
    total_val_loss = 0
    with torch.set_grad_enabled(False):
        for batch in tqdm(dev_loader):
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            labels = batch['labels'].to(device)
            outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
            if loss_weighted:
                loss = loss_function(outputs[1], labels)
            else:
                loss = outputs[0]
            total_val_loss += loss.item()/len(dev_loader)

            for logits in outputs[1].cpu().numpy():
                dev_preds.append(np.argmax(logits))

    y_true = dev_batch_labels
    y_pred = dev_preds
    target_names = label_mapping.keys()
    train_report = classification_report(train_labels, train_preds, target_names=target_names)
    report = classification_report(y_true, y_pred, target_names=target_names)
    val_f1 = f1_score(y_true, y_pred, average='macro')

    if val_f1 > best_val_f1:
        PATH = 'finetuned_models/' + model_name + '.pth'
        torch.save(model.state_dict(), PATH)
        model.save_pretrained(os.path.join('finetuned_berts/', model_name))
        best_val_f1 = val_f1
        count = 0
    else:
        count += 1

    print(train_report) 
    print(report)
    print("Epoch {}, Train Loss = {}, Val Loss = {}, Val F1 = {}, Best Val f1 = {}, stagnant = {}".format(epoch, total_train_loss, total_val_loss, val_f1, best_val_f1, count))
    if count == 5:
        print("No increase for 5 epochs, Stopping ...")
        break

Epoch 0
Train


  0%|                                                                                         | 0/2197 [00:00<?, ?it/s]C:\Users\cs\AppData\Local\Temp\ipykernel_17140\2961227109.py:9: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
C:\Users\cs\AppData\Local\Temp\ipykernel_17140\2961227109.py:15: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item['labels'] = torch.tensor(self.labels[idx])
100%|███████████████████████████████████████████████████████████████████████████| 2197/2197 [17:44:43<00:00, 29.08s/it]


Dev


100%|████████████████████████████████████████████████████████████████████████████████| 275/275 [36:39<00:00,  8.00s/it]


               precision    recall  f1-score   support

Not_offensive       0.84      0.52      0.64     25425
    not-Tamil       0.12      0.83      0.21      1454
    Offensive       0.41      0.46      0.44      8260

     accuracy                           0.52     35139
    macro avg       0.46      0.60      0.43     35139
 weighted avg       0.71      0.52      0.57     35139

               precision    recall  f1-score   support

Not_offensive       0.85      0.78      0.81      3193
    not-Tamil       0.44      0.83      0.57       172
    Offensive       0.49      0.54      0.52      1023

     accuracy                           0.73      4388
    macro avg       0.59      0.72      0.63      4388
 weighted avg       0.75      0.73      0.73      4388

Epoch 0, Train Loss = 0.779722643005343, Val Loss = 0.6495709014887159, Val F1 = 0.6346326604433172, Best Val f1 = 0.6346326604433172, stagnant = 0
Epoch 1
Train


...

  0%|                                                                                         | 0/2197 [00:00<?, ?it/s]C:\Users\cs\AppData\Local\Temp\ipykernel_17140\2961227109.py:9: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
C:\Users\cs\AppData\Local\Temp\ipykernel_17140\2961227109.py:15: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item['labels'] = torch.tensor(self.labels[idx])
100%|███████████████████████████████████████████████████████████████████████████| 2197/2197 [18:19:28<00:00, 30.03s/it]


Dev


100%|████████████████████████████████████████████████████████████████████████████████| 275/275 [40:55<00:00,  8.93s/it]


               precision    recall  f1-score   support

Not_offensive       0.89      0.69      0.78     25425
    not-Tamil       0.32      0.93      0.47      1454
    Offensive       0.48      0.65      0.56      8260

     accuracy                           0.69     35139
    macro avg       0.56      0.76      0.60     35139
 weighted avg       0.77      0.69      0.71     35139

               precision    recall  f1-score   support

Not_offensive       0.89      0.73      0.80      3193
    not-Tamil       0.40      0.89      0.55       172
    Offensive       0.50      0.69      0.58      1023

     accuracy                           0.72      4388
    macro avg       0.60      0.77      0.64      4388
 weighted avg       0.78      0.72      0.74      4388

Epoch 2, Train Loss = 0.4581421369708557, Val Loss = 0.5233237918144601, Val F1 = 0.6433796277862514, Best Val f1 = 0.6433796277862514, stagnant = 0
Epoch 3
Train


  0%|                                                                                         | 0/2197 [00:00<?, ?it/s]C:\Users\cs\AppData\Local\Temp\ipykernel_17140\2961227109.py:9: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
C:\Users\cs\AppData\Local\Temp\ipykernel_17140\2961227109.py:15: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item['labels'] = torch.tensor(self.labels[idx])
 77%|████████████████████████████████████████████████████████▎                | 1696/2197 [14:07:19<3:38:37, 26.18s/it]